<a href="https://colab.research.google.com/github/aap94/hello-world/blob/master/Project1_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MEMS1029 Project #1: Design of High Speed Shaft

Team 7: Sarah Fritzsche, Aarti Patel, Stephanie Manasterski, Nikolas Schunn

Project Goal: Tasked with designing the high-speed shaft for the transmission of a small industrial mixing apparatus. The purpose of the transmission is to convert the high-speed power output from the electric motor to the low-speed input for the mixing paddles. 

In [ ]:
#always input these first few lines
%matplotlib notebook 
import matplotlib.pyplot as plt 
import numpy as np 
import sympy as sym

In [ ]:
#define variables for shaft
Tin = 15 #torque input N*m
do = 0.09 #length from coupling to bearing in meters (center to center)
da = 0.065 #length from bearing1 to spur gear in meters (center to center)
db = 0.065 #length from spur gear to bearing2 in meters (center to center)
theta = 20*(np.pi/180) #pressure angle converted to radians
d1 = 0.01905 #diameter of shaft from coupling to bearing in meters
d2 = 0.03 #diameter of shaft from bearing to spur gear in meters
d3 = 0.025 #diameter of shaft from spur gear to bearing in meters
d4 = 0.01905 # diameter of shaft from second bearing towards paddles in meters
r_gear = 0.0675 # pitch diameter of the gear

## Shaft Physics
**Initial Force Analysis**

In [ ]:
# Gear force determined by torque 
F1 = Tin/(2*r_gear*np.cos(theta))
print('The forces at each gear contact are {:.2f} N in magnitude'.format(F1))

The forces at each gear contact are 118.24 N in magnitude


In [ ]:
#F components of spur gear
Fy = F1*(np.sin(theta)+np.cos(theta))
Fz = F1* (np.cos(theta) - np.sin(theta))
print ('Fy = {:.2f} N'.format(Fy))
print ('Fz = {:.2f} N'.format(Fz))

print ('F_totgear = {:.2f} N'.format(np.sqrt(Fz**2+Fy**2)))

#sum of moments in Y about A
Rby = (Fy*da)/(da+db)
print ('The Reaction force Rby = {:.2f} N'.format(Rby)) 

#sum of forces in Y
Ray = Fy-Rby
print ('The Reaction force Ray = {:.2f} N'.format(Ray))

#sum of moments in Z about A
Rbz = (Fz*da)/(da+db)
print ('The Reaction force Rbz = {:.2f} N'.format(Rbz))

#sum of forces in Z
Raz = Fz - Rbz
print ('The Reaction force Raz = {:.2f} N'.format(Raz))

Fy = 151.55 N
Fz = 70.67 N
F_totgear = 167.22 N
The Reaction force Rby = 75.78 N
The Reaction force Ray = 75.78 N
The Reaction force Rbz = 35.33 N
The Reaction force Raz = 35.33 N


In [ ]:
#calculate moments at Gear
MGz= (Rby*db) 
print('The Moment at the Gear about Z is MGz = {:.2f} Nm'.format(MGz))
MGy= (Rbz*db) 
print('The Moment at the Gear about Y is MGy = {:.2f} Nm'.format(MGy))

#calculate max moment 
MG = ((MGz**2)+(MGy**2))**0.5
print('The Max Moment at the Gear is MG = {:.2f} Nm'.format(MG))


The Moment at the Gear about Z is MGz = 4.93 Nm
The Moment at the Gear about Y is MGy = 2.30 Nm
The Max Moment at the Gear is MG = 5.43 Nm


**Gear Shaft Stress Analysis**

In [ ]:
#define stress concentration variables
shoulder_fillet = .0005 # m, radius of shoulder fillet 
D_d_1 = d3/d2 # D/d for A-15-7
r_d = shoulder_fillet/d2 # r/d for A-15-7
print('D_d for the gear = {:.2f}'.format(D_d_1))
print('r_d for the gear = {:.2f}'.format(r_d))
#Max (geometrically) allowed fillet at this corner: 1/4 * pi * r**2

#Torsional
ktG = 2.2 # from keyseat (worse than diameter change)
print('K_t for the gear = {:.2f}'.format(ktG))

# Axial loading:
ktsG = 3 # from keyseat (worse than diameter change)
print('K_ts for the gear = {:.2f}'.format(ktsG))

Sy = 530 # yield strength of SAE/AISI 1045 units in MPa (from shigley)
Sut = 630 # tensile strength of SAE/AISI 1045 units in MPa
SeTH = 0.5*Sut # Theoretical max fatigue strength in MPa
n = 4 # min factor of safety decided by design engineers
Tau_MG = 0 # defining variable for if statement
a = 4.51 # coeff for surface condition factor (shigleys) with MPa
b = -0.265 # exponent for surface condition factor (shigleys) with MPa

# Surface condition factor calculation for fatigue
ka = a*(Sut**b)

# Minimum radius on shaft
r_min = (n*4*MG/(Sy*(10**6)*np.pi))**(1/3)
d_min = r_min*2 # converting to diameter
#print('The minimum shaft diameter at the gear w/ respect to yield is {:.5f} meters'.format(d_min))
print('The minimum shaft diameter at the gear w/ respect to yield is {:.5f} mm'.format(d_min*1000)) # Slightly easier to read, no change of vars needed

# Defining worst bending stress location
if d2 > d3:
	r_s=d3/2
else:
	r_s=d2/2

# Smaller area at gear based on radius
A_s=np.pi*(r_s)**2

# Defining 2nd moment of area and polar moment of area for stress calculations
I = np.pi*((r_s)**4)/4 # 2nd moment of area in m^4
J=2*I # polar moment of area in m^4

# Maximum bending stress at the gear
SigmaMax=MG*r_s/I

#Defining torsional shear and transverse shear stresses for calculation at gear and bearing1 (A)
Tau_torsionGear = (Tin*r_s/J)/(10**6) # torsional shear stress in MPa
Tau_transGear=(4*(((Ray**(2))+(Raz**(2)))**0.5)/(3*A_s))/(10**6) # transverse shear stress in MPa
# Determining which scenario is worse (if transverse can be neglected)
if Tau_torsionGear > Tau_transGear: 
	Tau_MG = Tau_torsionGear # setting torsional shear stress max
	
	#Equivalent maximum alternating stress at the gear
	VonMises_stress=((((32*ktG*MG)/(np.pi*(r_s*2)**3))**2 + (3*((16*ktsG*Tin)/(np.pi*(r_s*2)**3))**2))**0.5)/(10**6) # MPa

	#printing Maximum von Mises
	print('The maximum von Mises stress in the shaft at the gear is {:.1f} MPa'.format(VonMises_stress))
 
  #Calculating FoS at gear w/ respect to DET yield and printing
	n_g = Sy/VonMises_stress
	print('The minimum factor of safety with respect to DET yield at the gear is {:.1f}'.format(n_g))

elif Tau_torsionGear > SigmaMax:
	Tau_MG = Tau_transGear
	SigmaMax = 0 # removes bending stress if it is not to be used
	print('Neglect Bending Stress')
else:
	Tau_maxG=Tau_torsionGear
	print('Neglect Transverse Shear')
 
# Calculate fatigue FoS
#Defining variables
q = 0.65 # notch sensitivity factor
qs = 0.7 # torsional sensitivity factor
kf_G = 1+(q*(ktG-1)) # Fatigue stress concentration # bending
kfs_G = 1+(qs*(ktsG-1)) # fatigue shear stress concentration 

# Calculating size factor for fatigue
kb_G = 1.24*((r_s*2*(10**3))**-0.107)

# Determining fatigue strength
Se_G = SeTH*kb_G*ka

# A value for goodman FoS
A_G = ((4*(kf_G*MG)**2)**0.5)/(10**6) # in MPa

# B value for goodman FoS
B_G  = ((3*(kfs_G*Tau_torsionGear)**2)**0.5)/(10**6) # in MPa

# Fatigue FoS with respect to goodman criteria
n_Gf = ((np.pi*(r_s*2)**3)/16)*(((A_G/Se_G)+(B_G/Sut))**(-1))
print('The fatigue factor of safety with respect to goodman criteria at the gear is {:.1f}'.format(n_Gf))

D_d for the gear = 0.83
r_d for the gear = 0.02
K_t for the gear = 2.20
K_ts for the gear = 3.00
The minimum shaft diameter at the gear w/ respect to yield is 7.47569 mm
The maximum von Mises stress in the shaft at the gear is 26.6 MPa
The minimum factor of safety with respect to DET yield at the gear is 19.9
The fatigue factor of safety with respect to goodman criteria at the gear is 26.0


**Bearing 1 Shaft Stress Analysis**

In [ ]:
#define stress concentration variables
shoulder_fillet = .0005 # m, radius of shoulder fillet 
D_d_1 = d2/d1 # D/d for A-15-7
r_d = shoulder_fillet/d1 # r/d for A-15-7
print('D_d for the gear = {:.2f}'.format(D_d_1))
print('r_d for the gear = {:.2f}'.format(r_d))

# Stress analysis at Bearing 1 change in diameter
kts1 = 2 #input using A-15-8 <- initially use these for Kf and Kfs

#Defining variables
qs = 0.7 # torsional sensitivity factor
kfs1 = 1+(qs*(kts1-1)) # fatigue shear stress concentration 

# Stress analysis at Bearing 1 at smaller shaft diameter (no stress concentration due to transverse shear only)
 
# Defining variables
A_b1 = np.pi*(d1**2)/4 # area of shaft at bearing 1 in in m^2
F_b1 = np.sqrt(Ray**2+Raz**2) # shear force at bearing 1 section in N
print('F_b1 = {:.1f} N'.format(F_b1)) #N
sigmaM_b1 = np.sqrt(3*((16*kfs1*(Tin)/(np.pi*d1**3))**2)) # midrange stress at Bearing 1 in MPa due to torsion
J_b = np.pi*((d1/2)**4)/2

# Tranverse shear stress
shear_b1 = 4*F_b1/(3*A_b1) # transverse different location
 
# Equivalent alternating stress calculation if using transverse
sigmaA_b1 = ((3*(shear_b1**2))**0.5) # in Pa
 
# Size factor for fatigue
kb_b1 = 1.24*((d1*(10**3))**-0.107) # with mm as d2 units
 
# Real fatigue strength calculation
Se_b1 = SeTH*ka*kb_b1
 
# Fatigue factor of safety with respect to modified goodman criteria
nf_b1 = (1/((sigmaM_b1/(Sut*10**6))))
#nf_b1 = ((np.pi*d1**3*Sut*10**6)/(sigmaM_b1))
# Printing results
print('The minimum fatigue factor of safety with respect to modified goodman criteria at bearing 1 is {:.1f}'.format(nf_b1))

D_d for the gear = 1.57
r_d for the gear = 0.03
F_b1 = 83.6 N
The minimum fatigue factor of safety with respect to modified goodman criteria at bearing 1 is 19.4


**Bearing 2 Shaft Stress Analysis**

In [ ]:
# Stress analysis at Bearing 2 at smaller shaft diameter (no stress concentration due to transverse shear only)
#define stress concentration variables
shoulder_fillet = .001 # m, radius of shoulder fillet 
D_d_1 = d3/d4 # D/d for A-15-7
r_d = shoulder_fillet/d4 # r/d for A-15-7
print('D_d for the gear = {:.2f}'.format(D_d_1))
print('r_d for the gear = {:.2f}'.format(r_d))

# Defining variables
A_b2 = np.pi*(d4**2)/4 # area of shaft at bearing 2 in in m^2
F_b2 = np.sqrt(Rby**2+Rbz**2) # shear force at bearing 2 section in N
sigmaM_b2 = 0 # midrange stress at Bearing 2 in MPa
print('F_b2 = {:.1f} N'.format(F_b2)) #N

# Transverse shear calculation (alternating stress but no other stresses present) <- different max location on cross-section than torsion and bending
shear_b2 = 4*F_b2/(3*A_b2)

# Equivalent alternating stress calculation
sigmaA_b2 = ((3*(shear_b2**2))**0.5)/(10**6) # in MPa

# Size factor for fatigue
kb_b2 = 1.24*((d4*(10**3))**-0.107) # with mm as d4 units

# Real fatigue strength calculation
Se_b2 = SeTH*ka*kb_b2

# Fatigue factor of safety with respect to modified goodman criteria
nf_b2 = 1/((sigmaA_b2/Se_b2)+(sigmaM_b2/Sut))
# Printing results
print('The minimum fatigue factor of safety with respect to modified goodman criteria at bearing 2 is {:.1f}'.format(nf_b2))

D_d for the gear = 1.31
r_d for the gear = 0.05
F_b2 = 83.6 N
The minimum fatigue factor of safety with respect to modified goodman criteria at bearing 2 is 343.8


**Coupling Keyseat Stress Concentration**

In [ ]:
# Defining variables
kts_C = 3.0 
Se_C=Se_b1*(10**6) # same section of shaft (Pa)

# Equivalent midrange stress due to torsional shear stress
sigmaM_C = np.sqrt(3*((16*kts_C*(Tin)/(np.pi*d1**3))**2))
print('The equivalent midrange stress due to torsional shear stress at the coupling keyseat is {:.1f} MPa'.format(sigmaM_C/(10**6)))

# Factor of safety at keyseat w/ respect to goodman criteria
n_C = 1/(sigmaM_C/(Sut*(10**6)))
print('Factor of Safety at keyseast with respect to goodman criteria is {:.1f}'.format(n_C))

The equivalent midrange stress due to torsional shear stress at the coupling keyseat is 57.4 MPa
Factor of Safety at keyseast with respect to goodman criteria is 11.0


## Bearing Analysis
**Radial Force Fatigue Life**

In [ ]:
# Code for bearing analysis
# Governing equation: F*L**(1/a) = C_10 * (k*L_10)**(1/a)

# Define constants
a = 3 # Depends on if we choose roller (10/3) or ball (3) bearing
w = 2000 # rotations/min
k = .33 # 95% efficiency expected
L_10 = 10**6 # 10% of bearing fail after 10^6 rotations
C_10Choose1 = 7784 # C_10 of chosen bearing 1 N
C_10Choose2 = 7784 # C_10 of chosen bearing 2 N
# F: calculated above this

# note that the 60 in the equation above converts minutes to hours
t_hrs = 5 #machine is ran for 5 hrs/day
t_days = 300 # machine is ran 300 days per year
t_years = 10 # machine should run for 10 years

# Radial loads at each bearing
F_b1 = np.sqrt(Ray**2+Raz**2)
# F_b2 = np.sqrt(Rby**2+Rbz**2) <- reiterated from prior code

L = w * 60 * t_hrs * t_days * t_years # conversion from rot/min to total rotations in lifetime
print('Required bearing life L = {:.2f} * 10^9 rotations'.format(L/(10**9)))

C_10_1 = F_b1 * (L/(k*L_10)) ** (1/a)
print('Minimum required catalog load rating C_10 = {:.2f} N for bearing 1'.format(C_10_1))
# FoS of bearing requirements based on chosen bearing
n_b1 = C_10Choose1/C_10_1
print('"Factor of Safety" of bearing 1 with chosen bearing is {:.2f}'.format(n_b1))

# Repeat for bearing #2
C_10_2 = F_b2 * (L/(k*L_10)) ** (1/a)
print('Minimum required catalog load rating C_10 = {:.2f} N for bearing 2'.format(C_10_2))
# FoS of bearing requirements based on chosen bearing
n_b2 = C_10Choose2/C_10_2
print('"Factor of Safety" of bearing 2 with chosen bearing is {:.2f}'.format(n_b2))

Required bearing life L = 1.80 * 10^9 rotations
Minimum required catalog load rating C_10 = 1471.78 N for bearing 1
"Factor of Safety" of bearing 1 with chosen bearing is 5.29
Minimum required catalog load rating C_10 = 1471.78 N for bearing 2
"Factor of Safety" of bearing 2 with chosen bearing is 5.29
